
# Linear Algebra with Python — Chapter 2: Matrix‑Vector Equations

**Focus:** practical NumPy/`pandas` solutions to `A x = b`, minimal math exposition, toy WNBA case study.  
**Libraries:** `numpy`, `pandas`, `matplotlib`.


In [1]:

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

print("Matplotlib:", matplotlib.__version__)
np.set_printoptions(suppress=True, linewidth=140)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)


Matplotlib: 3.10.6
NumPy: 2.3.3
Pandas: 2.3.3



## Motivation for Solving Matrix‑Vector Equations

Many ML tasks boil down to **solving** (or approximately solving) `A x = b`:
- Least squares (linear regression): `A` = features, `x` = weights, `b` = targets.
- Physics/graphics/optim: constraints as linear equations.
- Sports ratings: infer team strengths from game outcomes.


## The Meaning of $A x = b$

### From a Mathematics Perspective

$A x = b$ is a **system of linear equations**: the vector $b$ should be expressible as a **linear combination of the columns of $A$**, with weights given by $x$.  

- If $A$ is square and invertible → one unique solution.  
- If $A$ has more equations than unknowns → usually no exact solution, so we approximate (least squares).  
- If $A$ has fewer equations → infinitely many solutions.

- Geometrically: $x$ are the coordinates of $b$ in the space spanned by $A$'s columns.

### From a Programming Perspective

Programatically, we try to find the vector, $x$, which satisfies the previous equation. There are algorithms to converge to a solution, if one exists (Jacobi method, Gauss-Seidel, etc.) In NumPy, `A @ x` computes the matrix-vector product. Therefore, solving numerically with Numpy we have these scenarios:

- `np.linalg.solve(A, b)` → exact solution (square, full rank).
- `np.linalg.lstsq(A, b)` → least squares (overdetermined).
- `np.linalg.pinv(A) @ b` → minimum-norm solution (underdetermined).

In [3]:

# Geometric view: columns of A combine (with coefficients x) to reach b
A = np.array([[1., 2.],
              [3., 1.],
              [0., 1.]])
b = np.array([5., 7., 1.])

# Solve least squares (overdetermined: 3 eq, 2 unk)
x_hat, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
print("A:\n", A)
print("b:", b)

print("")
print("x_hat (least-squares):", x_hat)
print("residuals:", residuals)
print("rank:", rank)
print("singular values:", s)

# Check reconstruction and residual vector r = b - A x_hat
r = b - A @ x_hat
print("")
print("||r||_2 =", np.linalg.norm(r))


A:
 [[1. 2.]
 [3. 1.]
 [0. 1.]]
b: [5. 7. 1.]

x_hat (least-squares): [1.88571429 1.42857143]
residuals: [0.25714286]
rank: 2
singular values: [3.65857415 1.6170452 ]

||r||_2 = 0.5070925528371099


In [18]:
# What does Ax = b mean, again?
computed_b = A @ x_hat
diff = computed_b - b
print("b:         ", b)
print("computed b:", computed_b)
print("difference:", diff)
print("||diff||_2:", np.linalg.norm(diff))
print("% diff:    ", [f"{round(float((diff[i] / b[i]) * 100), 2)} %" for i in range(b.size)])

b:          [5. 7. 1.]
computed b: [4.74285714 7.08571429 1.42857143]
difference: [-0.25714286  0.08571429  0.42857143]
||diff||_2: 0.5070925528371099
% diff:     ['-5.14 %', '1.22 %', '42.86 %']


In [20]:
# in previous cell, third component has very high relative error
# but overall component-wise errors are low:
r = b - computed_b
rel_residual = np.linalg.norm(r) / np.linalg.norm(b)
rmse = np.sqrt(np.mean(r**2))
mae = np.mean(np.abs(r))

print(f"||r||/||b|| = {rel_residual:.4f}")
print(f"RMSE        = {rmse:.4f}")
print(f"MAE         = {mae:.4f}")


||r||/||b|| = 0.0586
RMSE        = 0.2928
MAE         = 0.2571


### System Conditioning 

$cond(A) = ||A||_2 * ​||A−1||_2​$

- If cond(A) ≈ 1: the system is well-conditioned — small errors in 𝐴 or 𝑏 cause small errors in 𝑥.
- If cond(A) is large (say 10^6 or higher): the system is ill-conditioned — tiny perturbations in data may cause huge changes in the solution.
- If cond(A) = ∞: A is singular (not invertible).

In [ ]:
condA = np.linalg.cond(A)
print("cond(A) =", condA)  # large -> sensitive to noise/scaling

cond(A) = 2.262505797398426


## Exploring WNBA Data (toy case study)


We'll build a **toy** 2017‑style WNBA dataset: teams, games, home/away scores.  
We'll then compute **Massey ratings**, a classic least‑squares method.


In [26]:

teams = ["LVA", "MIN", "SEA", "LAL"]  # toy subset of teams
games = pd.DataFrame([
    ("LVA", "MIN", 80, 75),
    ("SEA", "LAL", 70, 85),
    ("MIN", "SEA", 92, 88),
    ("LAL", "LVA", 77, 82),
    ("SEA", "LVA", 79, 81),
    ("MIN", "LAL", 90, 70),
    ("LAL", "SEA", 88, 83),
    ("MIN", "LVA", 84, 86),
], columns=["home","away","home_pts","away_pts"])

games["diff"] = games["home_pts"] - games["away_pts"]
games


,home,away,home_pts,away_pts,diff
0,LVA,MIN,80,75,5
1,SEA,LAL,70,85,-15
2,MIN,SEA,92,88,4
3,LAL,LVA,77,82,-5
4,SEA,LVA,79,81,-2
5,MIN,LAL,90,70,20
6,LAL,SEA,88,83,5
7,MIN,LVA,84,86,-2


## Understanding the Massey Matrix

In [ ]:

team_index = {t:i for i,t in enumerate(teams)}
n_teams = len(teams)
n_games = len(games)

C = np.zeros((n_games, n_teams))
d = games["diff"].to_numpy(dtype=float)

for k,(h,a) in enumerate(zip(games["home"], games["away"])):
    C[k, team_index[h]] =  1.0
    C[k, team_index[a]] = -1.0

M = C.T @ C
b_vec = C.T @ d

print("Teams:", teams)
print("\nC (first 5 rows):\n", C[:5])
print("\nM = C^T C:\n", M)
print("\nRight-hand side b = C^T d:\n", b_vec)
print("\nrank(M):", np.linalg.matrix_rank(M))


## Why is a Matrix Not Invertible?


Typical reasons:
- Duplicate/linearly dependent rows or columns (rank deficiency).
- Not square.
- Determinant = 0 (singular).

Massey’s `M` is singular because adding a constant to all ratings does not change differences.  
Fix: replace the last equation with a **sum‑to‑zero** constraint: `sum(r_i) = 0`.


## Understanding a Linear System's Three Outcomes

In [ ]:

A_over = np.random.default_rng(0).standard_normal((5, 3))
b_over = np.random.default_rng(1).standard_normal(5)
x_over, *_ = np.linalg.lstsq(A_over, b_over, rcond=None)

A_exact = np.array([[2., 1.],[1., 1.]])
b_exact = np.array([4., 3.])
x_exact = np.linalg.solve(A_exact, b_exact)

A_under = np.random.default_rng(2).standard_normal((2, 4))
b_under = np.random.default_rng(3).standard_normal(2)
x_under = np.linalg.pinv(A_under) @ b_under

print("Overdetermined x_hat:", x_over)
print("Exactly determined x:", x_exact)
print("Underdetermined min-norm x:", x_under)


## Understanding the Massey Matrix (adjusted)

In [ ]:

M_adj = M.copy()
b_adj = b_vec.copy()

M_adj[-1, :] = 1.0  # sum-to-zero constraint
b_adj[-1] = 0.0

ratings = np.linalg.solve(M_adj, b_adj)
ratings_table = pd.DataFrame({"team": teams, "rating": ratings}).sort_values("rating", ascending=False)
ratings_table


## Inverting the Massey Matrix (demo)

In [ ]:

M_inv_demo = np.linalg.inv(M_adj)
ratings_via_inv = M_inv_demo @ b_adj
np.allclose(ratings, ratings_via_inv)


## Solving Matrix‑Vector Equations (practice)

In [ ]:

rng = np.random.default_rng(42)
A = rng.standard_normal((100, 3))
true_x = np.array([2.0, -1.0, 0.5])
b = A @ true_x + 0.1 * rng.standard_normal(100)

x_ls, *_ = np.linalg.lstsq(A, b, rcond=None)

lam = 1e-1
x_ridge = np.linalg.solve(A.T @ A + lam * np.eye(A.shape[1]), A.T @ b)

print("Least-squares x_hat:", x_ls)
print("Ridge x_hat:", x_ridge)


## An Analogy with Regular Algebra

In [ ]:

a = 3.0
b_scalar = 12.0
x_scalar = b_scalar / a

A = np.array([[3., 1.],
              [0., 2.]])
b_vec = np.array([12., 6.])
x_vec = np.linalg.solve(A, b_vec)

print("Scalar:", x_scalar)
print("Vector:", x_vec)


## 2017 WNBA Ratings! (toy) & Who Was the Champion?

In [ ]:

ratings_table.reset_index(drop=True, inplace=True)
ratings_table


In [ ]:

champ = ratings_table.iloc[0]
print("Champion (toy):", champ["team"], "rating:", float(champ["rating"]))


## Other Considerations for Matrix‑Vector Equations


- **Conditioning:** large condition numbers → unstable solutions. Consider scaling and regularization.  
- **Outliers:** robust losses (not covered here) instead of pure least squares.  
- **Constraints:** add equations (like sum‑to‑zero) or use optimization with constraints.


## Other Methods for Matrix‑Vector Equations


- **QR factorization** (behind `lstsq`)  
- **SVD** (behind `pinv`)  
- **Iterative solvers** for large sparse systems (CG, GMRES) — not demoed here.


## Alternatives to the Regular Matrix Inverse

In [ ]:

rng = np.random.default_rng(0)
A = rng.standard_normal((5,5))
b = rng.standard_normal(5)

x_solve = np.linalg.solve(A, b)
x_inv = np.linalg.inv(A) @ b
print("||x_solve - x_inv||:", np.linalg.norm(x_solve - x_inv))

B = np.array([[1., 2., 3.],
              [2., 4., 6.],
              [1., 0., 1.]])
y = np.array([1., 2., 0.])
x_pinv = np.linalg.pinv(B) @ y
print("rank(B):", np.linalg.matrix_rank(B))
print("pinv solution:", x_pinv)
print("residual norm:", np.linalg.norm(B @ x_pinv - y))


## Visual: Residuals Histogram (from LS example)

In [ ]:

rng = np.random.default_rng(42)
A = rng.standard_normal((100, 3))
true_x = np.array([2.0, -1.0, 0.5])
b = A @ true_x + 0.1 * rng.standard_normal(100)
x_ls, *_ = np.linalg.lstsq(A, b, rcond=None)
res = b - A @ x_ls

plt.figure(figsize=(5,3))
plt.hist(res, bins=15)
plt.title("Residuals (least squares)")
plt.xlabel("residual")
plt.ylabel("count")
plt.show()
